## Day 21

https://adventofcode.com/2020/day/21

In [1]:
import collections

In [2]:
import aocd

In [3]:
test_data = """
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
"""

In [4]:
foods = aocd.get_data(day=21, year=2020).splitlines()
# foods = test_data.splitlines()[1:]
len(foods)

34

In [5]:
def parse_food(data):
    for food in data:
        ingredients, allergens = food.split(' (contains ')
        allergens = allergens[:-1].split(', ')
        ingredients = ingredients.split(' ')
        yield ingredients, allergens

In [6]:
def process_food(data):
    maybe = {}
    all_ingredients = collections.Counter()
    for ingredients, allergens in parse_food(data):
        all_ingredients.update(ingredients)
        for allergen in allergens:
            if allergen in maybe:
                maybe[allergen] &= set(ingredients)
            else:
                maybe[allergen] = set(ingredients)
    return all_ingredients, maybe

### Solution to Part 1

In [7]:
def count_safe_ingredients(data):
    ingredient_count, maybe_allergens = process_food(data)
    exclude = set(item for s in maybe_allergens.values() for item in s)
    return [
        (ingredient, count)
        for ingredient, count in ingredient_count.items()
        if ingredient not in exclude
    ]

In [8]:
sum(ingredient[1] for ingredient in count_safe_ingredients(foods))

1930

### Solution to Part 2

In [9]:
def get_ingredients(data):
    all_ingredients, unsure = process_food(data)
    all_ingredients = set(all_ingredients.keys())
    safe_ingredients = set(x[0] for x in count_safe_ingredients(data))
    return all_ingredients, safe_ingredients, unsure

In [10]:
def identify_allergens(data):
    all_ingredients, safe_ingredients, unsure = get_ingredients(data)
    to_remove = collections.deque()
    identified = dict()
    for allergen, candidates in unsure.items():
        if len(candidates) == 1:
            to_remove.append(allergen)
            identified[allergen] = next(iter(candidates))
    while to_remove:
        identified_allergen = to_remove.popleft()
        ingredient = next(iter(unsure[identified_allergen]))
        for allergen in unsure:
            if allergen != identified_allergen:
                unsure[allergen].discard(ingredient)
            if len(unsure[allergen]) == 1 and allergen not in identified:
                to_remove.append(allergen)
                identified[allergen] = next(iter(unsure[allergen]))
    return identified

In [11]:
allergens = identify_allergens(foods)
allergens

{'dairy': 'spcqmzfg',
 'shellfish': 'spql',
 'sesame': 'xbdh',
 'peanuts': 'bltrbvz',
 'fish': 'dzqlq',
 'eggs': 'rpf',
 'wheat': 'bltzkxx',
 'nuts': 'pflk'}

In [12]:
','.join(value for key, value in sorted(allergens.items()))

'spcqmzfg,rpf,dzqlq,pflk,bltrbvz,xbdh,spql,bltzkxx'